In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Meshing RVE function

In [ ]:
def mesh(x,y):

    # returns [XYZ, CON, DOF] 
    # XYZ - array of nodal coordinates within RVE [nel x 2]
    # CON - array of node numbers for elements (linear QUADS [nel x 4])
    # DOF - array of element DOFs (4-node (linear) quadrilateral element => [nel x 8]); 
    
    # number of nodes and element in the RVE
    nnodesx = len(x)                     # number of horizontal nodes
    nnodesy = len(y)                    # number of vertical nodes 
    nelx = nnodesx-1                       # number of horizontal elements
    nely = nnodesy-1                       # number of vertical elements
    nnodes = nnodesx*nnodesy               # total number of nodes    

    # GENERATE COORDINATES OF NODES 'XYZ'
    XYZ = np.zeros((nnodes,2))
    for i in range(nnodesy):
        for j in range(nnodesx):
            XYZ[j+i*nnodesx, :] = [x[j],y[i]]

    # NODE NUMBERS FOR ELEMENTS 
    nel = nelx*nely                              # total number of elements in RVE
    CON = np.zeros((nel,4), dtype=int)           # [nel*4] array of node number for each element
    for i in range(nely):                        # loop over elements in the vertical direction 
        for j in range(nelx):                    # loop over elements in the horizontal direction 
            # element 'el' and corresponding node numbers
            CON[j+i*nelx, :] = [j+i*nnodesx, j+i*nnodesx+1,j+(i+1)*nnodesx+1, j+(i+1)*nnodesx] 

    # Global DOF for each element (4-node (linear) quadrilateral element)
    DOF = np.zeros((nel,2*4), dtype=int)
    for i in range(nel):
        # defines single row of DOF for each element 'i'
        DOF[i,:] = [CON[i,0]*2, CON[i,1]*2-1, CON[i,1]*2, CON[i,1]*2+1,CON[i,2]*2, CON[i,2]*2+1, CON[i,3]*2, CON[i,3]*2+1]
        
    return XYZ, CON, DOF

# Create Mesh

In [ ]:
# Composite material with 2 phases

# Mesh coordinates in x and y directions
x = [0, 1, 2, 3]
y = [0, 1, 2, 3]
nxnodes = len(x)
nynodes = len(y)

#generate mesh
XYZ, CON, DOF = mesh(x,y)
ndofs=2*len(XYZ)
th=1

In [ ]:
# Plot mesh
plt.plot(XYZ[:, 0], XYZ[:, 1], 'sk')
for i in range(len(CON)):
    plt.fill(XYZ[CON[i, :], 0], XYZ[CON[i, :], 1], edgecolor='k', fill=False)

# Plane Strain

In [ ]:
def plane_strain(E,nu):
    
    D=np.zeros((3*len(E),3))                                           #elasticity matrix - DIM: [3*element number X 3]
    
    # Your code here
    raise NotImplementedError
    
    return D

# Apply Boundary Constraints (Fixed corner nodes)

In [ ]:
# Function which returns global DOFs given fixed corner nodes, and a list of internal left boundary nodes
def BCcorner_fun(nxnodes,nynodes,ndofs):
    
    # Identify corner nodes
    # YOUR CODE HERE
    corner0 = ...
    corner1 = ...
    corner2 = ...
    corner3 = ...
    corners = np.array([corner0,corner1,corner2,corner3])

    # Corner DOFs
    cornerDOFs = np.r_[corners*2,corners*2+1]

    # Get array of DOFs after removing corners
    BCcorner = np.arange(ndofs)
    BCcorner = np.setdiff1d(BCcorner,cornerDOFs)
    
    return BCcorner

# Strain-displacement matrix 

In [ ]:
def dispstrain_B(xyze,xi,eta):
    
    natcoord = np.array([[-1, 1, 1, -1],[-1, -1, 1, 1]])    #natural nodal coordinates of a quad element

    # derivatives of shape functions w.r.t. natural coordinates 
    dNdnat = np.zeros((2,4))
    # YOUR CODE HERE - Implement shape function derivatives

    # element Jacobian matrix
    Jmat = np.dot(dNdnat,xyze)
    J = np.linalg.det(Jmat)                          #determinant of the Jacobian
                
    JmatInv = np.linalg.inv(Jmat)                   #inverse of the Jacobian matrix
    dNdx = np.dot(JmatInv,dNdnat)                   #effectively: Jmat^-1 * dNdna
                
    #displacement-strain matrix 
    #linear QUAD element
    dsB=np.zeros((3,8))                  #[3 strain components X 8 DOFs]
    # YOUR CODE HERE - Implement displacement-strain matrix

    return dsB, J

# Element stiffness matrix

In [ ]:
def keval(xyze,De,th):
    
    ke = np.zeros((8,8))                                # create element stiffness matrix (array)                    

    a = 1/(np.sqrt(3))                                  # location of Gauss points (in natural coordinates)
    w = 1                                               # weights
    Gauss = np.array([[-a, a, a, -a],[-a, -a, a, a]])   # Gauss points matrix
        
    for i in range(4):                                # introduce natural coordinates 
        xi = Gauss[0,i]                                 # natural coordinate - horizontal  
        eta = Gauss[1,i]                                # natural coordinate - vertical 
        
        dsB, J = dispstrain_B(xyze,xi,eta);           # evaluate dsB matrix and Jacobian

        # YOUR CODE HERE - Update element stiffness matrix
        ke += ...
        
    return ke 

# Global stiffness matrix K

In [ ]:
def K_matrix(XYZ,CON,DOF,D,th,BCcorner):
    
    nel = len(CON)                        #number of elements
    ndof = 2*len(XYZ)                     #number of DOFs

    K = np.zeros((ndof,ndof))

    for i in range(nel):
        id = DOF[i,:]                      # IDs of DOFs
        xyze = XYZ[CON[i,:], :]             # element coordinates
        De = D[3*i:3*i+3, 0:3]              # elasticity matrix

        ke = keval(xyze,De,th)             # call function evaluating element stiffness matrix

        # YOUR CODE HERE - Update global stiffness matrix
        raise NotImplementedError

    K = K[np.ix_(BCcorner,BCcorner)] # Pin corners
    
    return K

# Deformation Plot

In [ ]:
def deformation_plot(XYZ,CON,d):
    ccc1=np.array(XYZ[:,0])
    ccc2=np.array(d[0:len(d):2]).reshape(-1)
    ccc= np.array(ccc1+ccc2) 
    
    ddd1=np.array(XYZ[:,1])
    ddd2=np.array(d[1:len(d):2]).reshape(-1)
    ddd= np.array(ddd1+ddd2)
    
    figure = plt.figure()
    plt.plot(XYZ[:,0], XYZ[:, 1],'sk', markersize='10')
    plt.plot(XYZ[:,0] + d[0:len(d):2].reshape(-1), XYZ[:,1] + d[1:len(d):2].reshape(-1), 'or',markersize='10')
    plt.title("Deformed RVE")
    
    for i in range(len(CON)):
        plt.fill(XYZ[CON[i, :], 0], XYZ[CON[i, :], 1], edgecolor='k', fill=False)
        plt.fill(XYZ[CON[i, :], 0] + ccc2[(CON[i, :])], XYZ[CON[i, :], 1] + ddd2[(CON[i, :])], edgecolor='r', fill=False)

# PBC Transformation Matrix

In [ ]:
def T_matrix(XYZ,BCcorner):

    # Number of nodes
    nnodes=len(XYZ) 

    # Boundary nodes
    small = 1e-10
    tid1=np.where(abs(XYZ[:,0]-np.amax(XYZ[:,0])) < small)[0] # Boundary 1 (RIGHT) - MASTER
    tid2=np.where(abs(XYZ[:,0]-np.amin(XYZ[:,0]))< small)[0] # Boundary 2 (LEFT) - SLAVE
    tid3=np.where(abs(XYZ[:,1]-np.amax(XYZ[:,1]))< small)[0] # Boundary 3 (TOP) - MASTER
    tid4=np.where(abs(XYZ[:,1]-np.amin(XYZ[:,1]))< small)[0] # Boundary 4 (BOTTOM) - SLAVE  

    # Exclude corners from slave node boundaries
    tids2=tid2[1:len(tid2)-1]
    tids4=tid4[1:len(tid4)-1]

    # Get number of slave nodes
    ns2nodes = len(tids2)
    ns4nodes = len(tids4)
    nsnodes = ns2nodes + ns4nodes

    # Create G matrix
    G=np.zeros((2*nsnodes,2*nnodes),dtype=int)          #create an array [DOFs no of slave nodes x DOFs no of all nodes in RVE]

    # loop over slave nodes
    for i in range(nsnodes):

        # First do left boundary
        if i < ns2nodes:
            
            # slave coefficients
            # YOUR CODE HERE - Populate G matrix components with slave node coefficients
            slavenode = ...
    
            # master coefficients
            # YOUR CODE HERE - Populate G matrix components with master node coefficients
            masternode = ...

        # Then top boundary
        else:
            
            # slave coefficients
            # YOUR CODE HERE - Populate G matrix components with slave node coefficients
            slavenode = ...
    
            # master coefficients
            # YOUR CODE HERE - Populate G matrix components with master node coefficients
            masternode = ...

    # Construct slave and master DOFs
    # Slaves
    slavedofs=np.zeros(2*nsnodes,dtype=int);        #array [n x 1] containing total DOF number of slave nodes
    b = np.r_[2*tids4,2*tids2]
    slavedofs[0:2*nsnodes-1:2] = b
    slavedofs[1:2*nsnodes:2] = b + 1

    # Masters, remove slaves from bound corner DOFs
    masterdofs = np.setdiff1d(BCcorner,slavedofs)
    nmnodes = len(masterdofs) // 2

    # Master and slave G matrices
    Gs=G[:,slavedofs]
    Gm=G[:,masterdofs]

    # Transformation matrix
    T=np.zeros((2*nnodes,2*nmnodes),dtype=int)

    # YOUR CODE HERE - Implement transformation matrix
    raise NotImplementedError

    # Remove fixed corners
    T=T[BCcorner,:]
    
    return T

# Element load vector

In [ ]:
def reval(xyze,De,th,eps):
                
    re=np.zeros((8,1))                                  # create element load vector                    

    a=1/(np.sqrt(3))                                  # location of Gauss points (in natural coordinates)
    w=1                                               # weights
    Gauss = np.array([[-a, a, a, -a],[-a, -a, a, a]]) # Gauss points matrix
        
    for i in range(4):                                # introduce natural coordinates 
        xi=Gauss[0,i]                                 # natural coordinate - horizontal  
        eta=Gauss[1,i]                                # natural coordinate - vertical 
        
        dsB, J = dispstrain_B(xyze,xi,eta)            # evaluate dsB matrix and Jacobian
    
        re += ...
        
    return re 

# Global load vector

In [ ]:
def r_vector(XYZ, CON, DOF, D, th, eps, BCcorner):
    
    nel=len(CON)                        #number of elements
    ndof=2*len(XYZ)                     #number of DOFs

    r=np.zeros((ndof,1))                  #create load vector

    for i in range(nel):
        id=DOF[i,:]                      # IDs of DOFs
        xyze=XYZ[CON[i,:],:]             # element coordinates
        De=D[3*i:3*i+3,0:3]              # elasticity matrix

        re = reval(xyze,De,th,eps)  # call function evaluating element stiffness matrix
    
        r[np.ix_(id)] += re         # global load vector

    r = -r[np.ix_(BCcorner)] # Constrain corners
    
    return r

# Average stress

In [ ]:
def sigmaHomo_vector(XYZ,CON,DOF,D,eps,d):
    
    nel=len(CON)                                            # number of elements    
    
    re=np.zeros((8,1))                                  # create element load vector                    

    a=1/(np.sqrt(3))                                    # location of Gauss points (in natural coordinates)
    w=1                                                 # weights
    Gauss = np.array([[-a, a, a, -a],[-a, -a, a, a]])   # Gauss points matrix

    sigma = np.zeros((3,1))                             # stress (Voigt notation)
    vol = 0
    
    for i in range(nel):
        id = DOF[i,:]                                     # IDs of DOFs
        xyze = XYZ[CON[i,:],:]                            # element coordinates
        De = D[3*i:3*i+3,0:3]                             # elasticity matrix
        de = d[id]
        
        for j in range(4):                                # introduce natural coordinates 
            xi=Gauss[0,j]                                 # natural coordinate - horizontal  
            eta=Gauss[1,j]                                # natural coordinate - vertical 
        
            dsB, J = dispstrain_B(xyze,xi,eta)            # evaluate dsB matrix and Jacobian
       
            sigmaGauss = ...                  # stresses at j-th Gauss point within i-th element
            
            sigma += sigmaGauss*J*w               # sum of the stresses over all Gauss points in i-th element

            vol += J                                # sum of the integration point volumes 
                
    sigmaHomo = sigma/vol                                    # vol = RVE volume 
    
    return sigmaHomo

# Compute Homogenisation of 11 Component of Elasticity Matrix

Function takes variable inputs for Young's modulii, Poison's ratios, and volume fraction

In [ ]:
def C11_homo(x,plot=False):

    # Extract inputs
    vFrac, E1, E2frac, nu1, nu2frac = x
    E2 = E1*E2frac
    nu2 = nu1*nu2frac

    # Create material property vectors
    nlayer = (nxnodes-1)*(nynodes-1)
    vFracs = np.linspace(0,1,nlayer+1)
    vFrac = vFracs[np.argmin(np.abs(vFracs-vFrac))]
    vFrac = Fraction(vFrac).limit_denominator()
    ndlayer = nlayer//vFrac.denominator*vFrac.numerator
    layerid = np.random.choice(nlayer,size=ndlayer,replace=False)
    E = np.ones(nlayer)*E1
    nu = np.ones(nlayer)*nu1
    E[layerid] = E2
    nu[layerid] = nu2
    
    # Elasticity matrix
    D = plane_strain(E,nu)
    
    # Get boundary conditions (fixed corners and force vector)
    BCcorner = BCcorner_fun(nxnodes,nynodes,ndofs)
    
    # Periodic boundary transform matrix
    T = T_matrix(XYZ,BCcorner)
    
    # Stiffness matrix
    K = K_matrix(XYZ,CON,DOF,D,th,BCcorner)
    
    # Modified stiffness matrix with PBCs
    Km = ...
    
    eps=np.zeros((3,1))                   # unit strain matrix (array in Voigt notation) 
    eps[0,0] = 1                          # given unit strain component 
    
    # load vector
    r = r_vector(XYZ, CON, DOF, D, th, eps, BCcorner)
 
    rm = ...                    # modified force vector with PBC
    dm = ...           # modified displacement components (for slave and master DOFs)
    dms = ...                    # displacement vector reduced problem solution

    # Global displacement vector
    d = np.zeros((ndofs,1))
    d[BCcorner] = dms
    
    # Calculate average stresses in the RVE
    sigmaHomo = sigmaHomo_vector(XYZ, CON, DOF, D, eps, d)
    C11 = sigmaHomo[0,0]

    # Plotting
    if plot:
        deformation_plot(XYZ,CON,d)

    return C11

# Test Case

In [ ]:
x = [4/9,300e9,0.333,0.2,0.5]
C11_homo(x,plot=True)

# Gaussian Process Regression

In [ ]:
from andvaranaut import *
import scipy.stats as st

In [ ]:
# Set prior distributions on inputs
vFrac = st.uniform(loc=0,scale=1) # Dispersed phase volume fraction
E1 = st.uniform(loc=200,scale=400) # E1
E2frac = st.uniform(loc=0.2,scale=0.5) # E2/E1
nu1 = st.uniform(loc=0.15,scale=0.2) # nu1
nu2frac = st.uniform(loc=0.2,scale=0.5) # nu2/nu1
priors = [vFrac,E1,E2frac,nu1,nu2frac]

In [ ]:
# Initialise GP object
g = GPMCMC(nx=5,ny=1,target=C11_homo,priors=priors,kernel='RBF',noise=True,verbose=True)

In [ ]:
# Evaluate samples from priors
g.sample(256)

In [ ]:
# Trim volume fraction data to fit discrete values allowed by 9 element system
vFracs = np.linspace(0,1,10)
g.x[:,0] = np.array([vFracs[np.argmin(np.abs(vFracs-i))] for i in g.x[:,0]])

In [ ]:
# Fit GP using PyMC
g.fit()

In [ ]:
# Check hyperparameters
print(f'Gaussian noise: {g.hypers["gv"]}')
print(f'Kernel noise: {g.hypers["kv"]}')
print(f'Kernel lengthscales: {g.hypers["l"]}')

In [ ]:
# Acquire train and test sets
g.train_test(training_frac=0.8)

# Plot test predicitions vs actual function
g.test_plots(xplots=False)

In [ ]:
# Add pre-processing of input and output data
xconrevs = [maxmin(g.x[:,i]) for i in range(g.nx)] # Scale x data to range [0,1] by max and min values
yconrevs = [meanstd(g.y[:,0])] # Scale y data by subtracting mean and dividing by standard deviation
g.change_conrevs(xconrevs=xconrevs,yconrevs=yconrevs)

In [ ]:
# Re-fit GP
g.fit()

In [ ]:
# Check hyperparameters
print(f'Gaussian noise: {g.hypers["gv"]}')
print(f'Kernel noise: {g.hypers["kv"]}')
print(f'Kernel lengthscales: {g.hypers["l"]}')

In [ ]:
# Check performance on same test set as previously
g.test_plots(xplots=False)

In [ ]:
# Predict at new x at volume fraction outwith our discrete limitations
xtest = np.array([[i.rvs() for i in priors]])
gpred, gpredvar = g.predict(xtest,return_var=True)

In [ ]:
# Get result at nearest discrete volume fraction for comparison
# and account for randomness with 100 realisations
evals = 100
ycloses = np.zeros(evals)
for i in range(evals):
    ycloses[i] = C11_homo(xtest[0])
yclose = np.mean(ycloses)
yclosestd = np.std(ycloses)

In [ ]:
# Compare results
with np.printoptions(precision=3, suppress=True):
    print(f'Test point is: {xtest[0]}')
    print(f'GP prediction is {gpred[0,0]:0.3f} +- {np.sqrt(gpredvar[0,0]):0.3f}')
    print(f'Result at nearest discrete volume fraction is {yclose:0.3f} +- {yclosestd:0.3f}')